## 連接雲端

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
folderPath = '/content/drive/MyDrive/交付檔案/【FinalCode】'
os.chdir(folderPath)
os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['最原始資料.xlsx',
 'DataSet',
 '門店資料_健身工廠.xlsx',
 'dfNewPartnerStore.csv',
 'poya2022-12-31.csv',
 'df3DAnalysis.xlsx',
 'dfFinal.csv',
 'distance_of_nearest_store.xlsx',
 'dfFinalPartner.csv',
 'dfFinal_Partner.csv',
 'dfFinal_Partner.gsheet',
 '爬蟲資料',
 'Codelist.gsheet',
 'finalCode.ipynb']

In [ ]:
import time # for sleep
import json # to parse json file 
import requests # request data from url
import pandas as pd # convert data into excel format
import numpy as np # deal with matrix and array 
from bs4 import BeautifulSoup # parse the html scripts
import random # for random retrieve number
!pip install -U googlemaps
import googlemaps # for google API
!pip install pygame
from typing import Tuple, List
from collections.abc import Iterable
from datetime import datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for googlemaps: filename=googlemaps-4.7.3-py3-none-any.whl size=40341 sha256=26abba69ffca5bd1068eaa6875a95a6f557cb8c745307e29198feb2da6e4a0be
  Stored in directory: /root/.cache/pip/wheels/e0/57/cc/329195e1c9e5b7d270fba4671c451d111d08f7b7ef56f8ec30
Successfully built googlemaps
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.8 MB 1.8 MB/s 


In [ ]:
# 請輸入自己的 google API key
key = "you google map api key"
# gmaps = googlemaps.Client(key=key)

# 程式執行戳記
date_of_today = datetime.now().strftime("%Y%m%d")

## Step 1 爬蟲

以寶雅為例

In [ ]:
def crawlerStore():  
  '''Return the List which contains the store of potential partner, but exclude west region and island'''
  print(f"Now {date_of_today}, Crawling")
  poyaStore = []
  poyaAddress = []
  poyaCity =[]
  poyaArea=[]
  #cityid = [2,3,4,8,14,15] 1-21
  for i in range(1,22):
    poyaURL =f'https://webapi.91app.com/webapi/LocationV2/GetLocationListByCity?cityId={i}&startIndex=0&maxCount=100&r=null&lang=zh-TW&shopId=40916'
    response = requests.get(poyaURL)
    response.json()
    poya = response.json()
    #citylist = ['新北市','台北市','桃園市','台中市','台南市','高雄市']
    for eachStore in poya['Data']['List']:
      #if eachStore['CityName'] in citylist:
        poyaStore.append(eachStore['Name'])
        poyaAddress.append(eachStore['Address'])
        poyaCity.append(eachStore['CityName'])
        poyaArea.append(eachStore['AreaName'])
  poyaList = pd.DataFrame({'storeName':poyaStore, 'Address':poyaAddress,'city':poyaCity,'area':poyaArea})
  # 排除這些縣市
  poyaListExcept = poyaList[~poyaList['city'].isin(['台東縣','花蓮縣','澎湖縣','金門縣'])]
  # 回傳排除外島跟東部縣市的縣市，如台北市、屏東市等
  return poyaListExcept


[連結文字](https://)Google Map API - from address to latitude&longitude

In [ ]:
!pip install geopy
from geopy.geocoders import Nominatim
from time import sleep
from random import randint
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from geopy.extra.rate_limiter import RateLimiter


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Step2. 利用政府資料幫助貼標土地使用分類

In [ ]:
!pip install fake-useragent
from fake_useragent import UserAgent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 3.0 MB/s 


In [ ]:
def findFullAddress(dataFrame: Iterable,cAddress: str,token: str,cookie: str,startNode: int,endNode: int) -> Iterable: 
  '''Input the DataFrame, which the place you want to find the complete address
  startNode: the index you want to start searching
  endNode: the index you want to stop searching
  '''
  city = []
  road = []
  label = []
  full_add = []
  district = []
  village  = []
  lng_list = []
  lat_list = []
  c_list = [] #citylist
  add_list = [] #addresslist
  for s in dataFrame[cAddress]:
    c_list.append(s[:3])
    add_list.append(s[3:])
  CMD = "GETADDRESS"
  # Cookie 需要自行更改，進入到網頁後，按下 F12，選取 network，往下拉會看到 Cookie，更改成出現的 Cookie 即可
  # referer 也需要自行更改有時候會換網域
  #https://luz.tcd.gov.tw/tcdgm/Default.aspx
  headers = {
  "x-requested-with": "XMLHttpRequest",
  "referer": "https://nsp.tcd.gov.tw/tcdgm/Default.aspx",
  "Cookie": cookie,
  "user-agent": ua.random,
  }
  
  for i in range(startNode, endNode+1):
    if i-startNode > 200:
      break
    print(i)
    VAL = add_list[i] # address
    COUNTY = c_list[i] 
    url = f"https://nsp.tcd.gov.tw/tcdgm/ws_data.ashx?&CMD={CMD}&VAL={VAL}&COUNTY={COUNTY}"
    print(f"Step1-Crawling:", url)
    # https://luz.tcd.gov.tw/tcdgm/ws_identify.ashx?token=U65yWjl1zDnIAUMdC3gzyljtSZsr7pF71BPZxQ316t3fhA7Ztfe6CK_zRiJ_b3YysH0hn7dH0FQlxWD_E4HdlA..&LAYERS=5%2C3%2C6&X=121.54446523313523&Y=25.047531690589974&Z=TWS
    data = requests.get(url, headers=headers).json()
    if(data):

      sleep_time = random.randint(11, 13)
      time.sleep(sleep_time) # temporary rest
      try:
        full_add.append(data["AddressList"][0][list(data["AddressList"][0].keys())[0]])
        city.append(data["AddressList"][0][list(data["AddressList"][0].keys())[1]])
        district.append(data["AddressList"][0][list(data["AddressList"][0].keys())[2]])
        village.append(data["AddressList"][0][list(data["AddressList"][0].keys())[3]])
        road.append(data["AddressList"][0][list(data["AddressList"][0].keys())[5]])
        lng_list.append(data["AddressList"][0][list(data["AddressList"][0].keys())[13]])
        lat_list.append(data["AddressList"][0][list(data["AddressList"][0].keys())[14]])
        lng = data["AddressList"][0]["X"]
        lat = data["AddressList"][0]["Y"]
      except(Exception) as error:
       print(error)
      # second step
      # token = "U65yWjl1zDnIAUMdC3gzyljtSZsr7pF71BPZxQ316t3fhA7Ztfe6CK_zRiJ_b3YysH0hn7dH0FQlxWD_E4HdlA.."
      url = f"https://nsp.tcd.gov.tw/tcdgm/ws_identify.ashx?token={token}&LAYERS=5%2C3%2C6&X={lng}&Y={lat}&Z=TWS"
      print(f"Step2-Crawling:", url)
      
      data = requests.get(url, headers=headers).json()
      
      sleep_time = random.randint(11, 13)
      time.sleep(sleep_time) # temporary rest
    
      try:
          label.append(data["5"]["features"][0]["properties"]["使用分區"])
      except(Exception) as error:
          print(error)
          print("Error occured while crawling:", url)
          label.append("error")    
      print(len(label), label)

    else:
      full_add.append("")
      city.append("")
      district.append("")
      village.append("")
      road.append("")
      lng_list.append("")
      lat_list.append("")    
  
  DataFrame = {
    "city":  city,
    "road":  road,
    "label": label,
    "full_add": full_add,
    "district": district,
    "village":  village,
    "lng_list": lng_list,
    "lat_list": lat_list,
  }
  newAddress = pd.DataFrame(DataFrame)
  # 回傳具有完整地址的DataFrame
  return newAddress
    



將住宅區、商業區轉換成二元變數
*   商業區：本組針對問卷中的提問「離學校/公司近」將商業區定義為包含許多學校以及公司的區域。
=>包含原商業區、文教區、工業區、行政區等
*   住宅區：本組針對問卷中的提問「離家近」將住宅區定義為包含許多住家的區域。
=>包含原住宅區、市場用地、工商綜合區、公園用地等
*   其他：包含原交通用地、娛樂區、鄉村區、農業區、醫療用地

In [ ]:
def judgeFloor(r):   
  if r['full_add'][-1] == "F":
    output = r['full_add'][-2]
  else :
    output = '1F'
  return output

In [ ]:
def judgeFloor1(r):   
  if r['address'][-1] == "F":
    output = r['address'][-2]
  else :
    output = '1F'
  return output

In [ ]:
def solveReplaceDistrict(dfPTStore: Iterable) -> Iterable:
  dfPTStore['labelNew'] = dfPTStore['label']
  dfDistrict = pd.read_excel("./DataSet/區域特性商業區住宅區字典.xlsx")
  business  = dfDistrict ["商業區"]
  business_list = business.values.tolist()
  residential = dfDistrict ["住宅區"]
  residential_list = residential.values.tolist()
  other  = dfDistrict ["其他區"]
  other_list = other.values.tolist()
  dfPTStore['labelNew'] = dfPTStore['labelNew'].replace(business_list,'商業區')
  dfPTStore['labelNew'] = dfPTStore['labelNew'].replace(residential_list,'住宅區')
  dfPTStore['labelNew'] = dfPTStore['labelNew'].replace(other_list,'其他區')
  dfdummy = pd.get_dummies(dfPTStore["labelNew"])
  dfPTStore1 = pd.concat((dfPTStore, dfdummy), axis=1)
  #是否一樓
  # floorScore_list = dfPTStore ["floor"].values.tolist()
  # dfPTStore['是否一樓'] = dfPTStore['floor'].replace(floorScore_list,'是否一樓')
  dfdummy1 = pd.get_dummies(dfPTStore["floor"])
  dfPTStore2 = pd.concat((dfPTStore1, dfdummy1), axis=1)
  return dfPTStore2

## Step3. 計算旅次密度

In [ ]:
def TripDensity(df1, city: str, district: str, afternoon_work: str, df2,
                 city_district: str, area: str) -> Iterable:
    """
    TripDensity() 用於製作「旅次密度」表格
    df1=旅次資料集，city=df1的縣市欄位，district=df1的鄉鎮市區欄位，afternoon_work=df1的平日午後旅次欄位，df2=人口密度資料集，city_district=df2的區域別欄位，area=df2的土地面積
    """
    df1.drop(index=0, inplace=True)
    df1["city"] = df1[city]
    df1["district"] = df1[district]
    df1["afternoon_work"] = df1[afternoon_work]
    # 合併city欄位和district欄位，得出city_district欄位
    df1["city_district"] = df1["city"] + df1["district"]
    df1 = df1[["city_district", "afternoon_work"]]

    df2["city_district"] = df2[city_district]
    df2["area"] = df2[area]
    df2 = df2[["city_district", "area"]]

    # 合併旅次資料集和人口密度資料集
    df3 = pd.merge(df1, df2, how="left")
    df3["trip_density"] = df3["afternoon_work"] / df3["area"]
    df3 = df3[["city_district", "trip_density"]]

    return df3

## Step4. 計算人口密度

In [ ]:
def PopulationDensity(df: Iterable, city_district: str, population_density: str) -> Iterable:
    """
    PopulationDensity() 用於製作「人口密度」表格
    df=人口密度資料集，city_district=區域別欄位，population_density=人口密度欄位
    
    """
    df["city_district"] = df[city_district]
    df["population_density"] = df[population_density]
    df = df[["city_district", "population_density"]]

    return df

## Step5. 計算供不應求

In [ ]:
"""
匯入「電信旅次」資料
取出縣市、區、平日晚上旅次欄位
匯入「SPX門市」資料
取出縣市欄位、鄉鎮市區欄位

各city_district的平日晚上旅次/全部city_district的平日晚上旅次總和，得出各city_district的平日晚上旅次比例
各city_district的SPX門市數/各city_district的SPX門市數總和，得出各city_district的SPX門市數比例
(平日晚上旅次比例-SPX門市數比例)*平日晚上旅次
"""


def ExcessDemand(df1, trip_city: str, trip_district: str,
                 evening_work_trip: str, df2, alias: str, spx_city: str,
                 spx_district: str) -> Iterable:
    """
    ExcessDemand() 用於製作供不應求的表格
    df1=電信旅次資料集，trip_city=df1的縣市欄位，trip_district=df1的鄉鎮市區欄位，evening_work_trip=df1的平日晚上旅次欄位，df2=SPX門市資料集，alias=df2的門市名稱欄位，spx_city=df2的縣市欄位，spx_district=df2的鄉鎮市區欄位
    """
    df1.drop(index=0, inplace=True)
    df1["trip_city"] = df1[trip_city]
    df1["trip_district"] = df1[trip_district]
    df1["evening_work_trip"] = df1[evening_work_trip]
    df1["city_district"] = df1["trip_city"] + df1["trip_district"]
    # 計算平日夜間旅次總合
    sum_trip = np.sum(df1["evening_work_trip"])
    # 計算每個city_district的平日夜間旅次比例
    df1["evening_trip_percentage"] = df1["evening_work_trip"].astype(
        float) / sum_trip
    df1 = df1[[
        "city_district", "evening_work_trip", "evening_trip_percentage"
    ]]

    df2["spx"] = df2[alias]
    df2["spx_city"] = df2[spx_city]
    df2["spx_district"] = df2[spx_district]
    df2["city_district"] = df2["spx_city"] + df2["spx_district"]
    # 計算總共有幾間SPX
    sum_spx_num = df2.spx.count()
    # 計算每個city_district的SPX數量
    df2 = df2.groupby(["city_district"
                       ])["spx"].count().reset_index(name="spx_num")
    # 計算每個city_district的SPX比例
    df2["spx_percentage"] = df2["spx_num"] / sum_spx_num
    df2 = df2[["city_district", "spx_percentage"]]

    df3 = pd.merge(df1, df2, how="left")
    df3["spx_percentage"].fillna(0, inplace=True)
    df3["excess_demand_percentage"] = df3["evening_trip_percentage"] - df3[
        "spx_percentage"]
    df3["excess_demand_trip"] = df3["excess_demand_percentage"] * df3[
        "evening_work_trip"]
    df3 = df3[["city_district", "excess_demand_trip"]]

    return df3

## Step6. 計算 3D 分群

In [ ]:
def ThreeDimension(df, trip_density: str, population_density: str, excess_demand_trip: str) -> Iterable:
  """
  ThreeDimension() 用於製作3D分群的表格
  df=資料集，trip_density=旅次密度欄位，population_density=人口密度欄位，excess_demand_trip=供不應求欄位
  """
  df["trip_density"] = df[trip_density]
  df["population_density"] = df[population_density]
  df["excess_demand_trip"] = df[excess_demand_trip]

  # 將3D分群所需欄位的數值進行百分比排序，並建立對應欄位
  df["district_rank"] = df["trip_density"].rank(pct=True)  # pct=True以百分比進行排序
  df["density_rank"] = df["population_density"].rank(pct=True)
  df["demand_rank"] = df["excess_demand_trip"].rank(pct=True)

  # 將district_rank由小到大轉換為1、2、3的標籤，並建立district_label欄位
  district_label = []
  for index, row in df.iterrows():  # 透過df.iterrows()對每一列進行迴圈
      i = row["district_rank"]  # 選定district_rank欄位的值進行迴圈
      if i <= 0.33:
          district_label.append("1")
      elif i <= 0.66:
          district_label.append("2")
      elif i <= 1:
          district_label.append("3")
  df["district_label"] = district_label

  # 將density_rank由小到大轉換為1、2、3的標籤，並建立density_label欄位
  density_label = []
  for index, row in df.iterrows():
      i = row["density_rank"]
      if i <= 0.33:
          density_label.append("1")
      elif i <= 0.66:
          density_label.append("2")
      elif i <= 1:
          density_label.append("3")
  df["density_label"] = density_label

  # 將demand_rank由小到大轉換為1、2、3的標籤，並建立demand_label欄位
  demand_label = []
  for index, row in df.iterrows():
      i = row["demand_rank"]
      if i <= 0.33:
          demand_label.append("1")
      elif i <= 0.66:
          demand_label.append("2")
      elif i <= 1:
          demand_label.append("3")
  df["demand_label"] = demand_label

  # 將district_labe、density_labe、demand_rank合併成3D_label
  df["3D_label"] = df["district_label"].astype(
      str) + df["density_label"] + df["demand_label"]

  return df

In [ ]:
def ThreeDimension1(df, trip_density: str, population_density: str, excess_demand: str) -> Iterable:
  """
  ThreeDimension() 用於製作3D分群的表格
  df=資料集，trip_density=旅次密度欄位，population_density=人口密度欄位，excess_demand=供不應求欄位
  """
  df["trip_density"] = df[trip_density]
  df["population_density"] = df[population_density]
  df["excess_demand"] = df[excess_demand]

  # 將3D分群所需欄位的數值進行百分比排序，並建立對應欄位
  df["district_rank"] = df["trip_density"].rank(pct=True)  # pct=True以百分比進行排序
  df["density_rank"] = df["population_density"].rank(pct=True)
  df["demand_rank"] = df["excess_demand"].rank(pct=True)

  # # 將district_rank由小到大轉換為1、2、3的標籤，並建立district_label欄位
  district_label = []
  for index, row in df.iterrows():  # 透過df.iterrows()對每一列進行迴圈
      i = row["district_rank"]  # 選定district_rank欄位的值進行迴圈
      if i <= 0.33:
          district_label.append("1")
      elif i <= 0.66:
          district_label.append("2")
      elif i <= 1:
          district_label.append("3")
  df["district_label"] = district_label

  # # 將density_rank由小到大轉換為1、2、3的標籤，並建立density_label欄位
  density_label = []
  for index, row in df.iterrows():
      i = row["density_rank"]
      if i <= 0.33:
          density_label.append("1")
      elif i <= 0.66:
          density_label.append("2")
      elif i <= 1:
          density_label.append("3")
  df["density_label"] = density_label

  # # 將demand_rank由小到大轉換為1、2、3的標籤，並建立demand_label欄位
  demand_label = []
  for index, row in df.iterrows():
      i = row["demand_rank"]
      if i <= 0.33:
          demand_label.append("1")
      elif i <= 0.66:
          demand_label.append("2")
      elif i <= 1:
          demand_label.append("3")
  df["demand_label"] = demand_label

  # # 將district_labe、density_labe、demand_rank合併成3D_label
  # df["3D_label"] = df["district_label"].astype(
  #     str) + df["density_label"] + df["demand_label"]

  return df

##Step6. 計算夥伴與蝦皮店到店的距離

In [ ]:
from numpy import sin, cos, arccos, pi

In [ ]:
def rad2deg(radians: float) -> float:
  """Input raidans, Return degree"""
  degrees = radians*180/pi
  return degrees

def deg2rad(degrees: float) -> float:
  """Input degree, Return radians"""
  radians = degrees*pi/180
  return radians

In [ ]:
def getDistanceBetweenPoints(latitude1: float, longitude1: float, latitude2: float, longitude2: float, unit = 'meters') -> float:
    """Input latitude and longitude of two locations and specify the unit of distance, Return distance between two locations"""
    theta = longitude1 - longitude2
    
    distance = 60*1.1515*rad2deg(
        # 反餘弦
        arccos(
            (sin(deg2rad(latitude1))*sin(deg2rad(latitude2))) + 
            (cos(deg2rad(latitude1))*cos(deg2rad(latitude2))  * cos(deg2rad(theta)))
        )
    )
    
    if unit == 'kilometers':
        return round(distance*1.609344, 2)
    if unit == 'meters':
        return round((distance*1.609344)*1000, 2)

In [ ]:
def calculateCloestDistanceBetweenPoints(partner: Iterable,spx: Iterable) -> Iterable:
  """Input the partner csv and output the excel file that contains the most closest distance between SPX and partner"""
  print(f"Now {date_of_today}, Calculating") 
  # read the file
  
  # partner = pd.read_excel(filepathPT)
  # spx = pd.read_excel(filepathSpx)
  # convert latitude and longitude to list
  lat_patner = list(partner["latitude"])
  lng_partner = list(partner["longitude"])

  lat_spx = list(spx["latitude"])
  lng_spx = list(spx["longitude"])
  spx_store = list(spx["alias"])
  # store the distace between the partner and the most closest SPX
  distance = []
  nearest_store = []

  # 合作夥伴
  for p in range(len(partner)):
      distance_dict2 = {}
      print(f"正在計算 合作夥伴: 第{p+1}間店址")
      # SPX
      for s in range(len(spx)):
          
          # 計算合作夥伴與全部SPX的距離
          distance_dict2[spx_store[s]] = getDistanceBetweenPoints(lat_patner[p], lng_partner[p], lat_spx[s], lng_spx[s], unit="meters")    
      # 為每個合作夥伴店址分別儲存距離清單
      for d in distance_dict2:
          if distance_dict2[d] == min(distance_dict2.values()):
              # 最近的店(公尺)
              print(d, distance_dict2[d])
              nearest_store.append(d)
              distance.append(distance_dict2[d])
  DataFrame = {
    "brand":list(partner["brand"]),
    "store": list(partner["store"]),
    "latitude":lat_patner,
    "longitude":lng_partner,
    "nearest_store": nearest_store,
    "distance(m)": distance,
  }

  file_name = f"distance_of_nearest_store"
  df = pd.DataFrame(DataFrame)
  # 將 DataFrame 輸出成 excel file
  df.to_excel(f"{file_name}.xlsx", index=False)
  return df

## Step7. 計算附加效益

In [ ]:
def calculate_additional_benefit(index1: int, index2: int, radius: int, store_type: str, API_key: str,  data: Iterable) -> Iterable:
  """Return DataFrame. Input the DataFrame as data. Given the store type that search how many store_type in a circle with radius 
  and get the score from index1 to index2."""  
  print(f"Now {date_of_today}, Calculating") 
  # step 1 create variables
  # input the google api key
  map_client = googlemaps.Client(API_key)
  
  # Name list of brands 
  brand = []
  # Name of store of brands
  storeName = []
  latitude = []
  longitude = []
  # how many store_type in the area, which radius is given
  shoplistCount = []
  rows_count = len(data.index) 

  for i in range(index1, index2):
    print(f"Crawling {store_type}, Index: {i}")

    # random sleep for not interrupting crawling
    if i % 20 == 0:
        sleep_time = random.randint(5, 8)
        print(f"Now sleep for {sleep_time} sec")
        time.sleep(sleep_time)
    
    # store the store_type 
    business_list = []
      
    brand.append(data['brand'][i])
    storeName.append(data['store'][i])
    latitude.append(data['latitude'][i])
    longitude.append(data['longitude'][i])

    response = map_client.places_nearby(
        location = (data['latitude'][i], data['longitude'][i]),
        keyword  = store_type,
        radius   = radius
    )
      
    business_list.extend(response.get('results'))
    # get the next 20 results
    next_page_token = response.get('next_page_token')

    while next_page_token:
        time.sleep(2)
        response = map_client.places_nearby(
            location= (data['latitude'][i], data['longitude'][i]),
            keyword = store_type,
            radius  = radius,
            page_token=next_page_token
        ) 

        business_list.extend(response.get('results'))
        next_page_token = response.get('next_page_token')

    df = pd.DataFrame(business_list)
    resultlength = len(df.index) 
    shoplistCount.append(resultlength)
  
  DataFrame = {
    "brand": brand,
    "store": storeName,
    "latitude":latitude,
    "longitude":longitude,
    store_type+"_"+str(radius): shoplistCount
  }
  dfResult = pd.DataFrame(DataFrame)

  return dfResult

## Step8. 計算總分

In [ ]:
def calculateTotalScore(data: Iterable,
            spx_distance_weight: float,
            floor_weight: float,
            house_weight: float,
            business_weight: float, 
            cvs_weight: float, 
            train_weight: float,
            bus_weight: float,
            radius: int,businessList:list)-> Iterable:
  
  scores = []
  data['distance_score'] = pd.qcut(data['distance(m)'], 4,labels = False)
  data['total_score'] = data['住宅區分數'].astype('float') *  house_weight + \
  data['商業區分數'].astype('float') *  business_weight + \
  data[f'{businessList[2]}_{radius}'].astype('float') *  cvs_weight + \
  data[f'{businessList[1]}_{radius}'].astype('float') *  train_weight + \
  data[f'{businessList[0]}_{radius}'].astype('float') *  bus_weight + \
  data['是否一樓'].astype('float64') *  floor_weight
  # data['total_score'] = scores

  return data

## Main Function

### **Step1爬蟲門市＿以寶雅範例**


In [ ]:
#Step1爬蟲門市＿以寶雅範例

# 建立新的 dataframe
dfPoyaStore = pd.DataFrame()
# 呼叫 爬蟲 function 回傳爬蟲完的 dataframe
dfPoyaStore= crawlerStore()
# 檔案名為：爬蟲的品牌 + 爬蟲當下的日期
file_name = 'poya'+ date_of_today
# 存下 csv file
dfPoyaStore.to_csv(f'./{file_name}.csv')
dfPoyaStore

Now 2022-12-31, Crawling


,storeName,Address,city,area
0,基隆成功店,基隆市仁愛區成功一路144號1樓,基隆市,仁愛區
1,寶雅基隆暖暖店,基隆市暖暖區暖暖街168號,基隆市,暖暖區
2,寶雅基隆新豐店,基隆市中正區新豐街460號B1,基隆市,中正區
3,寶雅基隆東岸店,基隆市仁愛區仁二路236號1樓(東岸廣場公車站正後方),基隆市,仁愛區
4,寶雅北投中央北店,台北市北投區中央北路四段532號1樓,台北市,北投區
...,...,...,...,...
344,寶雅宜蘭蘇澳店,宜蘭縣蘇澳鎮中山路一段277號,宜蘭縣,蘇澳鎮
345,寶雅羅東倉前店,宜蘭縣羅東鎮倉前路66號(羅東國中附近),宜蘭縣,羅東鎮
346,寶雅宜蘭礁溪店,宜蘭縣礁溪鄉中山路二段188號,宜蘭縣,礁溪鄉
347,寶雅宜蘭站前店,宜蘭縣宜蘭市光復路6.8.10.12號,宜蘭縣,宜蘭市


### **Step2** **利用讀取爬蟲門市資料貼標** 

In [ ]:
#step2 
#利用讀取爬蟲門市資料貼標 

# 隨機生成 網路爬蟲的 user-agent
ua = UserAgent()  
# 將怕蟲品牌的店點資訊以 dataframe 的資料結構讀入
dfCrawlerStore = pd.read_excel('./DataSet/Step2_爬蟲門店資料整理_20221130.xlsx')
dfCrawlerStore

,brand,store,city_district,city,district,village,road,floor,latitude,longitude,address
0,Tomods,北投石牌二店,台北市北投區,台北市,北投區,永明里,石牌路,1F,25.117289,121.516357,台北市北投區永明里3鄰石牌路二段93號
1,Tomods,捷運內湖店,台北市內湖區,台北市,內湖區,內湖里,成功路,1F,25.084081,121.594547,台北市內湖區內湖里25鄰成功路四段188號
2,Tomods,三井林口店,新北市林口區,新北市,林口區,南勢里,文化三路,1F,25.071241,121.363547,新北市林口區南勢里22鄰文化三路一段356號
3,Tomods,中山五常店,台北市中山區,台北市,中山區,行仁里,五常街,1F,25.064402,121.539213,台北市中山區行仁里1鄰五常街47號
4,Tomods,南港中信店,台北市南港區,台北市,南港區,三重里,經貿二路,1F,25.059222,121.615185,台北市南港區三重里13鄰經貿二路186號
...,...,...,...,...,...,...,...,...,...,...,...
2628,寶雅,潮州新生店,屏東縣潮州鎮,屏東縣,潮州鎮,新生里,新生路,1F,22.551863,120.538545,屏東縣潮州鎮新生里13鄰新生路117號
2629,寶雅,屏東潮州店,屏東縣潮州鎮,屏東縣,潮州鎮,新榮里,新生路,1F,22.548106,120.538158,屏東縣潮州鎮新榮里15鄰新生路42號
2630,寶雅,高雄林園店,高雄市林園區,高雄市,林園區,林園里,仁愛路,1F,22.505147,120.392640,高雄市林園區林園里1鄰仁愛路46號
2631,寶雅,屏東東港店,屏東縣東港鎮,屏東縣,東港鎮,新街里,中正路,1F,22.475441,120.460499,屏東縣東港鎮新街里19鄰中正路一段316號


In [ ]:
#token 需要自行更改，進入到網頁後，按下 F12，選取 network，往下拉會看到 Cookie，更改成出現的 Cookie 即可
#八小時內限制最多 300筆

token = "U65yWjl1zDnIAUMdC3gzynMv21qEtZMFoyeyttMR3_3uA8jFKd2qGaHia9MhPbRUrYKkhyZsfQcJyCSzu1DzYg.."
# 呼叫 爬取店址地址 function，傳入 dataframe, 爬取的目標（地址），token, 從第幾間店開始，到第幾間店結束
newAddress = findFullAddress(dfCrawlerStore,'address',token,0,200)
newAddress


0
Step1-Crawling: https://luz.tcd.gov.tw/tcdgm/ws_data.ashx?&CMD=GETADDRESS&VAL=北投區永明里3鄰石牌路二段93號&COUNTY=台北市
Step2-Crawling: https://luz.tcd.gov.tw/tcdgm/ws_identify.ashx?token=U65yWjl1zDnIAUMdC3gzynMv21qEtZMFoyeyttMR3_3uA8jFKd2qGaHia9MhPbRUrYKkhyZsfQcJyCSzu1DzYg..&LAYERS=5%2C3%2C6&X=121.516357&Y=25.117289&Z=TWS
1 ['第三種商業區']
1
Step1-Crawling: https://luz.tcd.gov.tw/tcdgm/ws_data.ashx?&CMD=GETADDRESS&VAL=內湖區內湖里25鄰成功路四段188號&COUNTY=台北市
Step2-Crawling: https://luz.tcd.gov.tw/tcdgm/ws_identify.ashx?token=U65yWjl1zDnIAUMdC3gzynMv21qEtZMFoyeyttMR3_3uA8jFKd2qGaHia9MhPbRUrYKkhyZsfQcJyCSzu1DzYg..&LAYERS=5%2C3%2C6&X=121.594547&Y=25.084081&Z=TWS
2 ['第三種商業區', '交通用地']
2
Step1-Crawling: https://luz.tcd.gov.tw/tcdgm/ws_data.ashx?&CMD=GETADDRESS&VAL=林口區南勢里22鄰文化三路一段356號&COUNTY=新北市
Step2-Crawling: https://luz.tcd.gov.tw/tcdgm/ws_identify.ashx?token=U65yWjl1zDnIAUMdC3gzynMv21qEtZMFoyeyttMR3_3uA8jFKd2qGaHia9MhPbRUrYKkhyZsfQcJyCSzu1DzYg..&LAYERS=5%2C3%2C6&X=121.363547&Y=25.071241&Z=TWS
3 ['第三種商業區', '交通用地', '

,city,road,label,full_add,district,village,lng_list,lat_list
0,臺北市,石牌路,第三種商業區,臺北市北投區永明里3鄰石牌路二段93號,北投區,永明里,121.516357,25.117289
1,臺北市,成功路,交通用地,臺北市內湖區內湖里25鄰成功路四段188號,內湖區,內湖里,121.594547,25.084081
2,新北市,文化三路,中心商業區,新北市林口區南勢里22鄰文化三路一段356號,林口區,南勢里,121.363547,25.071241
3,臺北市,五常街,第三之一種住宅區,臺北市中山區行仁里1鄰五常街47號,中山區,行仁里,121.539213,25.064402
4,臺北市,經貿二路,商業區,臺北市南港區三重里13鄰經貿二路186號,南港區,三重里,121.615185,25.059222
...,...,...,...,...,...,...,...,...
196,高雄市,明誠三路,第四種商業區,高雄市鼓山區龍子里4鄰明誠三路465號,鼓山區,龍子里,120.301100,22.660966
197,高雄市,美術東二路,特定第五種住宅區,高雄市鼓山區龍水里11鄰美術東二路7號,鼓山區,龍水里,120.287724,22.651675
198,屏東縣,中山路,商業區,屏東縣麟洛鄉麟趾村17鄰中山路391號,麟洛鄉,麟趾村,120.526225,22.651138
199,高雄市,建工路,第三種特定商業專用區,高雄市三民區灣勝里13鄰建工路568號,三民區,灣勝里,120.324366,22.650151


In [ ]:
#門市貼標及地址資料更新的結果

# 讀取具有地址與店點的 excel file
dfPTStore = pd.read_excel("./DataSet/Step2_潛在合作門市店址_20221130.xlsx")
dfPTStore

,brand,store,city_district,city,district,village,road,label,latitude,longitude,address
0,Tomods,北投石牌二店,台北市北投區,台北市,北投區,永明里,石牌路,第三種商業區,25.117289,121.516357,台北市北投區永明里3鄰石牌路二段93號
1,Tomods,捷運內湖店,台北市內湖區,台北市,內湖區,內湖里,成功路,交通用地,25.084081,121.594547,台北市內湖區內湖里25鄰成功路四段188號
2,Tomods,三井林口店,新北市林口區,新北市,林口區,南勢里,文化三路,中心商業區,25.071241,121.363547,新北市林口區南勢里22鄰文化三路一段356號
3,Tomods,中山五常店,台北市中山區,台北市,中山區,行仁里,五常街,第三之一種住宅區,25.064402,121.539213,台北市中山區行仁里1鄰五常街47號
4,Tomods,南港中信店,台北市南港區,台北市,南港區,三重里,經貿二路,商業區,25.059222,121.615185,台北市南港區三重里13鄰經貿二路186號
...,...,...,...,...,...,...,...,...,...,...,...
2628,寶雅,潮州新生店,屏東縣潮州鎮,屏東縣,潮州鎮,新生里,新生路,商業區,22.551863,120.538545,屏東縣潮州鎮新生里13鄰新生路117號
2629,寶雅,屏東潮州店,屏東縣潮州鎮,屏東縣,潮州鎮,新榮里,新生路,商業區,22.548106,120.538158,屏東縣潮州鎮新榮里15鄰新生路42號
2630,寶雅,高雄林園店,高雄市林園區,高雄市,林園區,林園里,仁愛路,商業區,22.505147,120.392640,高雄市林園區林園里1鄰仁愛路46號
2631,寶雅,屏東東港店,屏東縣東港鎮,屏東縣,東港鎮,新街里,中正路,住宅區,22.475441,120.460499,屏東縣東港鎮新街里19鄰中正路一段316號



將住宅區、商業區轉換成二元變數
*   商業區：本組針對問卷中的提問「離學校/公司近」將商業區定義為包含許多學校以及公司的區域。
=>包含原商業區、文教區、工業區、行政區等
*   住宅區：本組針對問卷中的提問「離家近」將住宅區定義為包含許多住家的區域。
=>包含原住宅區、市場用地、工商綜合區、公園用地等
*   其他：包含原交通用地、娛樂區、鄉村區、農業區、醫療用地

In [ ]:
# 新增 1F 的欄位，如果是一樓輸出 1F，不是 1F，則輸出原本的樓層
dfPTStore['floor'] = dfPTStore.apply(judgeFloor1,axis=1)
dfPTStore = solveReplaceDistrict(dfPTStore)

In [ ]:
# 重新命名 住宅區、其他區、商業區、1F的欄位名稱
dfPTStore.rename(columns={"住宅區": "住宅區分數", "其他區": "其他區分數","商業區": "商業區分數","1F": "是否一樓"}, inplace = True)
dfPTStore

,brand,store,city_district,city,district,village,road,label,latitude,longitude,address,floor,labelNew,住宅區分數,其他區分數,商業區分數,是否一樓
0,Tomods,北投石牌二店,台北市北投區,台北市,北投區,永明里,石牌路,第三種商業區,25.117289,121.516357,台北市北投區永明里3鄰石牌路二段93號,1F,商業區,0,0,1,1
1,Tomods,捷運內湖店,台北市內湖區,台北市,內湖區,內湖里,成功路,交通用地,25.084081,121.594547,台北市內湖區內湖里25鄰成功路四段188號,1F,其他區,0,1,0,1
2,Tomods,三井林口店,新北市林口區,新北市,林口區,南勢里,文化三路,中心商業區,25.071241,121.363547,新北市林口區南勢里22鄰文化三路一段356號,1F,商業區,0,0,1,1
3,Tomods,中山五常店,台北市中山區,台北市,中山區,行仁里,五常街,第三之一種住宅區,25.064402,121.539213,台北市中山區行仁里1鄰五常街47號,1F,住宅區,1,0,0,1
4,Tomods,南港中信店,台北市南港區,台北市,南港區,三重里,經貿二路,商業區,25.059222,121.615185,台北市南港區三重里13鄰經貿二路186號,1F,商業區,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2628,寶雅,潮州新生店,屏東縣潮州鎮,屏東縣,潮州鎮,新生里,新生路,商業區,22.551863,120.538545,屏東縣潮州鎮新生里13鄰新生路117號,1F,商業區,0,0,1,1
2629,寶雅,屏東潮州店,屏東縣潮州鎮,屏東縣,潮州鎮,新榮里,新生路,商業區,22.548106,120.538158,屏東縣潮州鎮新榮里15鄰新生路42號,1F,商業區,0,0,1,1
2630,寶雅,高雄林園店,高雄市林園區,高雄市,林園區,林園里,仁愛路,商業區,22.505147,120.392640,高雄市林園區林園里1鄰仁愛路46號,1F,商業區,0,0,1,1
2631,寶雅,屏東東港店,屏東縣東港鎮,屏東縣,東港鎮,新街里,中正路,住宅區,22.475441,120.460499,屏東縣東港鎮新街里19鄰中正路一段316號,1F,住宅區,1,0,0,1


### **Step3 計算旅次密度**

In [ ]:
dfCarrierPop = pd.read_excel("./DataSet/DAC_台灣各時段電信人口資料.xlsx")
dfPopDensity = pd.read_excel("./DataSet/DAC_人口密度.xlsx")

# 呼叫 計算旅次密度的函數，計算台灣各地區的旅次密度
dfTravelDensity = TripDensity(df1=dfCarrierPop,
                   city="COUNTY",
                   district="TOWN",
                   afternoon_work="AFTERNOON_WORK",
                   df2=dfPopDensity,
                   city_district="區域別",
                   area="土地面積")
dfTravelDensity

,city_district,trip_density
0,新北市板橋區,52611.128351
1,新北市三重區,53991.551756
2,新北市中和區,49198.205917
3,新北市永和區,72459.711575
4,新北市新莊區,47103.589975
...,...,...
363,金門縣烏坵鄉,0.0
364,連江縣南竿鄉,752.0375
365,連江縣北竿鄉,208.236364
366,連江縣莒光鄉,225.204255


### **Step 4 計算人口密度**


In [ ]:
#Step 4 計算人口密度
# dfPopDensity = pd.read_excel("./dataset/人口密度.xlsx")

# 呼叫計算人口密度函數，計算台灣各區的人口密度
dfPopDensity_CC = PopulationDensity(df=dfPopDensity, city_district="區域別", population_density="人口密度")
dfPopDensity_CC

,city_district,population_density
0,新北市板橋區,23888.0
1,新北市三重區,23395.0
2,新北市中和區,20162.0
3,新北市永和區,37622.0
4,新北市新莊區,21348.0
...,...,...
365,連江縣北竿鄉,303.0
366,連江縣莒光鄉,318.0
367,連江縣東引鄉,393.0
368,東沙群島,…


### **Step5計算供不應求**


In [ ]:
#Step5計算供不應求
dfCarrierPop2 = pd.read_excel("./DataSet/DAC_台灣各時段電信人口資料.xlsx")
dfSpxStore = pd.read_excel("./DataSet/DAC_蝦皮店到店地理資料_20221012.xlsx")

# 呼叫計算 SPX 在台各區域的需求與供應函數，若 > 0 則 供給不足需求；若 < 0 則 供給超過需求
dfExcessDemand = ExcessDemand(df1=dfCarrierPop2, trip_city="COUNTY", trip_district="TOWN",
                  evening_work_trip="EVENING_WORK", df2=dfSpxStore, alias="alias", spx_city="city",
                  spx_district="district")
dfExcessDemand

,city_district,excess_demand_trip
0,新北市板橋區,-22108.938864
1,新北市三重區,-5732.774899
2,新北市中和區,-8470.396928
3,新北市永和區,-2355.534068
4,新北市新莊區,-9723.839323
...,...,...
363,金門縣烏坵鄉,0.0
364,連江縣南竿鄉,0.68445
365,連江縣北竿鄉,0.047673
366,連江縣莒光鄉,0.014543


In [ ]:
# 加入 merge 人口密度、供不應求、旅次密度

# 將 店址資訊與旅次密度、人口密度、供不應求三個 dataframes 合併起來
dfExcessDemand = pd.merge(dfExcessDemand, dfPopDensity_CC, how="left", on=["city_district"])
dfExcessDemand = pd.merge(dfExcessDemand, dfTravelDensity, how="left", on=["city_district"])
dfExcessDemand["city_district"]  = dfExcessDemand["city_district"].apply(lambda x: x.replace("台","臺"))
dfExcessDemand["city_district"] = dfExcessDemand["city_district"].str.strip()
dfPTStore["city_district"]  = dfPTStore["city_district"].apply(lambda x: x.replace("台","臺"))
dfPTStore["city_district"] = dfPTStore["city_district"].str.strip()
dfPTStore = pd.merge(dfPTStore, dfExcessDemand, how="left", on=["city_district"])
dfPTStore

,brand,store,city_district,city,district,village,road,label,latitude,longitude,address,floor,labelNew,住宅區分數,其他區分數,商業區分數,是否一樓,excess_demand_trip,population_density,trip_density
0,Tomods,北投石牌二店,臺北市北投區,台北市,北投區,永明里,石牌路,第三種商業區,25.117289,121.516357,台北市北投區永明里3鄰石牌路二段93號,1F,商業區,0,0,1,1,4851.012772,4307.0,9375.879243
1,Tomods,捷運內湖店,臺北市內湖區,台北市,內湖區,內湖里,成功路,交通用地,25.084081,121.594547,台北市內湖區內湖里25鄰成功路四段188號,1F,其他區,0,1,0,1,10120.778886,8762.0,23889.068264
2,Tomods,三井林口店,新北市林口區,新北市,林口區,南勢里,文化三路,中心商業區,25.071241,121.363547,新北市林口區南勢里22鄰文化三路一段356號,1F,商業區,0,0,1,1,-522.670625,2301.0,5099.81792
3,Tomods,中山五常店,臺北市中山區,台北市,中山區,行仁里,五常街,第三之一種住宅區,25.064402,121.539213,台北市中山區行仁里1鄰五常街47號,1F,住宅區,1,0,0,1,24434.937481,15755.0,89095.901945
4,Tomods,南港中信店,臺北市南港區,台北市,南港區,三重里,經貿二路,商業區,25.059222,121.615185,台北市南港區三重里13鄰經貿二路186號,1F,商業區,0,0,1,1,2275.074368,5301.0,15925.965095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2628,寶雅,潮州新生店,屏東縣潮州鎮,屏東縣,潮州鎮,新生里,新生路,商業區,22.551863,120.538545,屏東縣潮州鎮新生里13鄰新生路117號,1F,商業區,0,0,1,1,186.787421,1262.0,2301.658611
2629,寶雅,屏東潮州店,屏東縣潮州鎮,屏東縣,潮州鎮,新榮里,新生路,商業區,22.548106,120.538158,屏東縣潮州鎮新榮里15鄰新生路42號,1F,商業區,0,0,1,1,186.787421,1262.0,2301.658611
2630,寶雅,高雄林園店,高雄市林園區,高雄市,林園區,林園里,仁愛路,商業區,22.505147,120.392640,高雄市林園區林園里1鄰仁愛路46號,1F,商業區,0,0,1,1,176.852143,2134.0,2923.12891
2631,寶雅,屏東東港店,屏東縣東港鎮,屏東縣,東港鎮,新街里,中正路,住宅區,22.475441,120.460499,屏東縣東港鎮新街里19鄰中正路一段316號,1F,住宅區,1,0,0,1,128.345065,1570.0,2994.998558


In [ ]:
dfPTStore

# 計算 旅次密度、人口密度、供不應求的中位數，並將中位數當作填入缺失值的數值
dfPTStore["excess_demand_trip"].fillna(dfPTStore['excess_demand_trip'].median(), inplace = True)
dfPTStore["population_density"].fillna(dfPTStore['population_density'].median(), inplace = True)
dfPTStore["trip_density"].fillna(dfPTStore['trip_density'].median(), inplace = True)
dfPTStore.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2633 entries, 0 to 2632
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   brand               2633 non-null   object 
 1   store               2633 non-null   object 
 2   city_district       2633 non-null   object 
 3   city                2633 non-null   object 
 4   district            2633 non-null   object 
 5   village             2633 non-null   object 
 6   road                2633 non-null   object 
 7   label               2633 non-null   object 
 8   latitude            2633 non-null   float64
 9   longitude           2633 non-null   float64
 10  address             2633 non-null   object 
 11  floor               2633 non-null   object 
 12  labelNew            2633 non-null   object 
 13  住宅區分數               2633 non-null   uint8  
 14  其他區分數               2633 non-null   uint8  
 15  商業區分數               2633 non-null   uint8  
 16  是否一樓  

### **Step 6 計算3D分群**


In [ ]:
# Step 6 計算3D分群
df = pd.read_excel("./DataSet/Step5_3DAnalysis.xlsx")
df3DAnalysis = ThreeDimension(df=dfPTStore, trip_density="trip_density", population_density="population_density", excess_demand_trip="excess_demand_trip")
df3DAnalysis


,brand,store,city_district,city,district,village,road,label,latitude,longitude,...,excess_demand_trip,population_density,trip_density,district_rank,density_rank,demand_rank,district_label,density_label,demand_label,3D_label
0,Tomods,北投石牌二店,臺北市北投區,台北市,北投區,永明里,石牌路,第三種商業區,25.117289,121.516357,...,4851.012772,4307.0,9375.879243,0.400114,0.370870,0.785986,2,2,3,223
1,Tomods,捷運內湖店,臺北市內湖區,台北市,內湖區,內湖里,成功路,交通用地,25.084081,121.594547,...,10120.778886,8762.0,23889.068264,0.668249,0.613179,0.915496,3,2,3,323
2,Tomods,三井林口店,新北市林口區,新北市,林口區,南勢里,文化三路,中心商業區,25.071241,121.363547,...,-522.670625,2301.0,5099.817920,0.223319,0.235473,0.289784,1,1,1,111
3,Tomods,中山五常店,臺北市中山區,台北市,中山區,行仁里,五常街,第三之一種住宅區,25.064402,121.539213,...,24434.937481,15755.0,89095.901945,0.944740,0.749905,0.969047,3,3,3,333
4,Tomods,南港中信店,臺北市南港區,台北市,南港區,三重里,經貿二路,商業區,25.059222,121.615185,...,2275.074368,5301.0,15925.965095,0.558868,0.449107,0.667869,2,2,3,223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2628,寶雅,潮州新生店,屏東縣潮州鎮,屏東縣,潮州鎮,新生里,新生路,商業區,22.551863,120.538545,...,186.787421,1262.0,2301.658611,0.103114,0.119445,0.468857,1,1,2,112
2629,寶雅,屏東潮州店,屏東縣潮州鎮,屏東縣,潮州鎮,新榮里,新生路,商業區,22.548106,120.538158,...,186.787421,1262.0,2301.658611,0.103114,0.119445,0.468857,1,1,2,112
2630,寶雅,高雄林園店,高雄市林園區,高雄市,林園區,林園里,仁愛路,商業區,22.505147,120.392640,...,176.852143,2134.0,2923.128910,0.144132,0.209837,0.452526,1,1,2,112
2631,寶雅,屏東東港店,屏東縣東港鎮,屏東縣,東港鎮,新街里,中正路,住宅區,22.475441,120.460499,...,128.345065,1570.0,2994.998558,0.155526,0.150589,0.443031,1,1,2,112


In [ ]:
df3DAnalysis.to_excel("./df3DAnalysis.xlsx") 


### **Step 6 計算夥伴與蝦皮店到店距離**


In [ ]:
# Step 6 計算夥伴與蝦皮店到店距離

partner = pd.read_excel("./DataSet/Step6_夥伴貼標3D.xlsx")
spx = pd.read_excel("./DataSet/DAC_蝦皮店到店地理資料_20221012.xlsx")

# 呼叫 計算合作夥伴各店址與最近的蝦皮店到店的距離
dfNearStore = calculateCloestDistanceBetweenPoints(partner,spx)


串流輸出內容已截斷至最後 5000 行。
正在計算 合作夥伴: 第134間店址
桃園中山店 507.55
正在計算 合作夥伴: 第135間店址
樹林東榮店 298.35
正在計算 合作夥伴: 第136間店址
八德介壽店 406.37
正在計算 合作夥伴: 第137間店址
中壢環中店 211.77
正在計算 合作夥伴: 第138間店址
中壢慈惠店 125.5
正在計算 合作夥伴: 第139間店址
鶯歌中正店 607.8
正在計算 合作夥伴: 第140間店址
平鎮廣泰店 287.32
正在計算 合作夥伴: 第141間店址
八德中正店 487.58
正在計算 合作夥伴: 第142間店址
楊梅瑞梅店 223.41
正在計算 合作夥伴: 第143間店址
湖口溪北店 69.0
正在計算 合作夥伴: 第144間店址
龍潭東龍店 239.66
正在計算 合作夥伴: 第145間店址
竹北文義二店 279.9
正在計算 合作夥伴: 第146間店址
竹北福興店 220.71
正在計算 合作夥伴: 第147間店址
新竹民生店 262.34
正在計算 合作夥伴: 第148間店址
新竹中華店 403.05
正在計算 合作夥伴: 第149間店址
新竹中山店 775.95
正在計算 合作夥伴: 第150間店址
梧棲四維店 970.88
正在計算 合作夥伴: 第151間店址
沙鹿成功店 1640.61
正在計算 合作夥伴: 第152間店址
豐原惠陽店 78.03
正在計算 合作夥伴: 第153間店址
大雅民生二店 996.03
正在計算 合作夥伴: 第154間店址
潭子中山店 735.08
正在計算 合作夥伴: 第155間店址
台中福瑞店 410.02
正在計算 合作夥伴: 第156間店址
北屯僑孝店 398.81
正在計算 合作夥伴: 第157間店址
台中中清二店 358.79
正在計算 合作夥伴: 第158間店址
美廉社 台中山西店 514.36
正在計算 合作夥伴: 第159間店址
南屯黎明店 1137.35
正在計算 合作夥伴: 第160間店址
台中中清店 286.82
正在計算 合作夥伴: 第161間店址
台中十甲店 602.71
正在計算 合作夥伴: 第162間店址
南屯大墩店 259.26
正在計算 合作夥伴: 第163間店址
美廉社 太平永豐店 518.0
正在計算 合作夥伴: 第1

In [ ]:
dfNearStore

,brand,store,latitude,longitude,nearest_store,distance(m)
0,Tomods,北投石牌二店,25.117289,121.516357,北投石牌店,477.08
1,Tomods,捷運內湖店,25.084081,121.594547,內湖金龍店,289.83
2,Tomods,三井林口店,25.071241,121.363547,林口民族店,326.29
3,Tomods,中山五常店,25.064402,121.539213,松山復興店,530.21
4,Tomods,南港中信店,25.059222,121.615185,南港忠孝店,719.28
...,...,...,...,...,...,...
2628,寶雅,潮州新生店,22.551863,120.538545,鳳山過勇店,18495.54
2629,寶雅,屏東潮州店,22.548106,120.538158,鳳山過勇店,18554.69
2630,寶雅,高雄林園店,22.505147,120.392640,小港永義店,8261.96
2631,寶雅,屏東東港店,22.475441,120.460499,小港永義店,15401.89


In [ ]:
df3DAnalysis

,brand,store,city_district,city,district,village,road,label,latitude,longitude,...,excess_demand_trip,population_density,trip_density,district_rank,density_rank,demand_rank,district_label,density_label,demand_label,3D_label
0,Tomods,北投石牌二店,台北市北投區,台北市,北投區,永明里,石牌路,第三種商業區,25.117289,121.516357,...,4851.012772,4307.0,9375.879243,0.400114,0.370870,0.785986,2,2,3,223
1,Tomods,捷運內湖店,台北市內湖區,台北市,內湖區,內湖里,成功路,交通用地,25.084081,121.594547,...,10120.778886,8762.0,23889.068264,0.668249,0.613179,0.915496,3,2,3,323
2,Tomods,三井林口店,新北市林口區,新北市,林口區,南勢里,文化三路,中心商業區,25.071241,121.363547,...,-522.670625,2301.0,5099.817920,0.223319,0.235473,0.289784,1,1,1,111
3,Tomods,中山五常店,台北市中山區,台北市,中山區,行仁里,五常街,第三之一種住宅區,25.064402,121.539213,...,24434.937481,15755.0,89095.901945,0.944740,0.749905,0.969047,3,3,3,333
4,Tomods,南港中信店,台北市南港區,台北市,南港區,三重里,經貿二路,商業區,25.059222,121.615185,...,2275.074368,5301.0,15925.965095,0.558868,0.449107,0.667869,2,2,3,223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2628,寶雅,潮州新生店,屏東縣潮州鎮,屏東縣,潮州鎮,新生里,新生路,商業區,22.551863,120.538545,...,186.787421,1262.0,2301.658611,0.103114,0.119445,0.468857,1,1,2,112
2629,寶雅,屏東潮州店,屏東縣潮州鎮,屏東縣,潮州鎮,新榮里,新生路,商業區,22.548106,120.538158,...,186.787421,1262.0,2301.658611,0.103114,0.119445,0.468857,1,1,2,112
2630,寶雅,高雄林園店,高雄市林園區,高雄市,林園區,林園里,仁愛路,商業區,22.505147,120.392640,...,176.852143,2134.0,2923.128910,0.144132,0.209837,0.452526,1,1,2,112
2631,寶雅,屏東東港店,屏東縣東港鎮,屏東縣,東港鎮,新街里,中正路,住宅區,22.475441,120.460499,...,128.345065,1570.0,2994.998558,0.155526,0.150589,0.443031,1,1,2,112


In [ ]:
# 透過 left join 合併 3D 分群法與合作夥伴距離最近蝦皮店到店的距離
dfFinal = pd.merge(df3DAnalysis, dfNearStore, how="left", on=["brand","store","latitude","longitude"])
dfFinal.to_csv('./dfFinal.csv')
dfFinal

,brand,store,city_district,city,district,village,road,label,latitude,longitude,...,trip_density,district_rank,density_rank,demand_rank,district_label,density_label,demand_label,3D_label,nearest_store,distance(m)
0,Tomods,北投石牌二店,台北市北投區,台北市,北投區,永明里,石牌路,第三種商業區,25.117289,121.516357,...,9375.879243,0.400114,0.370870,0.785986,2,2,3,223,北投石牌店,477.08
1,Tomods,捷運內湖店,台北市內湖區,台北市,內湖區,內湖里,成功路,交通用地,25.084081,121.594547,...,23889.068264,0.668249,0.613179,0.915496,3,2,3,323,內湖金龍店,289.83
2,Tomods,三井林口店,新北市林口區,新北市,林口區,南勢里,文化三路,中心商業區,25.071241,121.363547,...,5099.817920,0.223319,0.235473,0.289784,1,1,1,111,林口民族店,326.29
3,Tomods,中山五常店,台北市中山區,台北市,中山區,行仁里,五常街,第三之一種住宅區,25.064402,121.539213,...,89095.901945,0.944740,0.749905,0.969047,3,3,3,333,松山復興店,530.21
4,Tomods,南港中信店,台北市南港區,台北市,南港區,三重里,經貿二路,商業區,25.059222,121.615185,...,15925.965095,0.558868,0.449107,0.667869,2,2,3,223,南港忠孝店,719.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2628,寶雅,潮州新生店,屏東縣潮州鎮,屏東縣,潮州鎮,新生里,新生路,商業區,22.551863,120.538545,...,2301.658611,0.103114,0.119445,0.468857,1,1,2,112,鳳山過勇店,18495.54
2629,寶雅,屏東潮州店,屏東縣潮州鎮,屏東縣,潮州鎮,新榮里,新生路,商業區,22.548106,120.538158,...,2301.658611,0.103114,0.119445,0.468857,1,1,2,112,鳳山過勇店,18554.69
2630,寶雅,高雄林園店,高雄市林園區,高雄市,林園區,林園里,仁愛路,商業區,22.505147,120.392640,...,2923.128910,0.144132,0.209837,0.452526,1,1,2,112,小港永義店,8261.96
2631,寶雅,屏東東港店,屏東縣東港鎮,屏東縣,東港鎮,新街里,中正路,住宅區,22.475441,120.460499,...,2994.998558,0.155526,0.150589,0.443031,1,1,2,112,小港永義店,15401.89


### **Step7 計算附加效益**

In [ ]:
dfPTStoreTag = pd.read_excel("./DataSet/Step6_夥伴貼標3D.xlsx")
#type you google api key
API_KEY = ''
#calculate_additional_benefit(dataframe start, dataframe end, radius distance, store_type, google API_key, data)
# the number of [ "學校", "公車站", "火車站","便利商店"]   
ltShoptype = [ "公車站", "火車站","便利商店"]
radius_list =[100,200,300,400,500,600] 

# dfFinal_North = pd.DataFrame()
# 計算附加效益，以100-600公尺為半徑的公車站、火車站、便利商店數量
for radius in radius_list:
  print(str(radius)+"m")
  for index,i in enumerate(ltShoptype) :
    dfStoreNum = calculate_additional_benefit( 0,5, radius, i,API_KEY, dfPTStoreTag)

    # 將總表與附加效益表合併
    dfPTStoreTag = pd.merge(dfPTStoreTag, dfStoreNum, how="left", on=["brand","store","latitude","longitude"])
dfPTStoreTag

100m
Now 2022-12-31, Calculating
Crawling 公車站, Index: 0
Now sleep for 8 sec
Crawling 公車站, Index: 1
Crawling 公車站, Index: 2
Crawling 公車站, Index: 3
Crawling 公車站, Index: 4
Now 2022-12-31, Calculating
Crawling 火車站, Index: 0
Now sleep for 6 sec
Crawling 火車站, Index: 1
Crawling 火車站, Index: 2
Crawling 火車站, Index: 3
Crawling 火車站, Index: 4
Now 2022-12-31, Calculating
Crawling 便利商店, Index: 0
Now sleep for 6 sec
Crawling 便利商店, Index: 1
Crawling 便利商店, Index: 2
Crawling 便利商店, Index: 3
Crawling 便利商店, Index: 4
200m
Now 2022-12-31, Calculating
Crawling 公車站, Index: 0
Now sleep for 7 sec
Crawling 公車站, Index: 1
Crawling 公車站, Index: 2
Crawling 公車站, Index: 3
Crawling 公車站, Index: 4
Now 2022-12-31, Calculating
Crawling 火車站, Index: 0
Now sleep for 5 sec
Crawling 火車站, Index: 1
Crawling 火車站, Index: 2
Crawling 火車站, Index: 3
Crawling 火車站, Index: 4
Now 2022-12-31, Calculating
Crawling 便利商店, Index: 0
Now sleep for 6 sec
Crawling 便利商店, Index: 1
Crawling 便利商店, Index: 2
Crawling 便利商店, Index: 3
Crawling 便利商店, Index: 4
30

,brand,store,city_district,city,district,village,road,floor,label,label_new,...,便利商店_300,公車站_400,火車站_400,便利商店_400,公車站_500,火車站_500,便利商店_500,公車站_600,火車站_600,便利商店_600
0,Tomods,北投石牌二店,台北市北投區,台北市,北投區,永明里,石牌路,1F,第三種商業區,商業區,...,13.0,13.0,1.0,15.0,17.0,1.0,20.0,19.0,1.0,24.0
1,Tomods,捷運內湖店,台北市內湖區,台北市,內湖區,內湖里,成功路,1F,交通用地,交通用地,...,0.0,14.0,0.0,11.0,14.0,0.0,13.0,16.0,0.0,0.0
2,Tomods,三井林口店,新北市林口區,新北市,林口區,南勢里,文化三路,1F,中心商業區,商業區,...,19.0,33.0,1.0,20.0,34.0,2.0,0.0,39.0,2.0,29.0
3,Tomods,中山五常店,台北市中山區,台北市,中山區,行仁里,五常街,1F,第三之一種住宅區,住宅區,...,0.0,16.0,0.0,0.0,26.0,1.0,0.0,27.0,0.0,0.0
4,Tomods,南港中信店,台北市南港區,台北市,南港區,三重里,經貿二路,1F,商業區,商業區,...,0.0,16.0,3.0,12.0,18.0,4.0,14.0,20.0,9.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2628,寶雅,潮州新生店,屏東縣潮州鎮,屏東縣,潮州鎮,新生里,新生路,1F,商業區,商業區,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2629,寶雅,屏東潮州店,屏東縣潮州鎮,屏東縣,潮州鎮,新榮里,新生路,1F,商業區,商業區,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2630,寶雅,高雄林園店,高雄市林園區,高雄市,林園區,林園里,仁愛路,1F,商業區,商業區,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2631,寶雅,屏東東港店,屏東縣東港鎮,屏東縣,東港鎮,新街里,中正路,1F,住宅區,住宅區,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Step8 計算總分**

### 標準化

In [ ]:
addtional_values = dfFinalPartnerShop.columns
addtional_values

Index(['brand', 'store', 'city_district', 'city', 'district', 'village',
       'road', 'nearest_spx', 'distance(m)', 'distance_score', 'floor', '是否一樓',
       '3D_label', 'label', 'label_重貼標', '住宅區', '商業區', '其他', '公車站_100',
       '火車站_100', 'CVS_100', '公車站_200', '火車站_200', 'CVS_200', '公車站_300',
       '火車站_300', 'CVS_300', '公車站_400', '火車站_400', 'CVS_400', '公車站_500',
       '火車站_500', 'CVS_500', '公車站_600', '火車站_600', 'CVS_600', 'latitude',
       'longitude', 'address'],
      dtype='object')

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 讀取 Excel file
dfFinalPartnerShop = pd.read_excel("./DataSet/Step8_合作夥伴最終版.xlsx")

# 額外拉出，要標準化的欄位
addtional_values = ['公車站_600','火車站_600','CVS_600',\
                        '公車站_500','火車站_500','CVS_500',\
                        '公車站_400','火車站_400','CVS_400',\
                        '公車站_300','火車站_300','CVS_300',\
                        '公車站_200','火車站_200','CVS_200',\
                        '公車站_100','火車站_100','CVS_100']
X = dfFinalPartnerShop[addtional_values] 
# 將各欄位，標準化為介於 0 - 1 之間的數值
scaler = MinMaxScaler(feature_range=(0, 1)).fit(X)
X_scaled = scaler.transform(X)
dfFinalPartnerShop[addtional_values] = X_scaled

# 設定計算合作夥伴店址分數的權重
# take the radius "300m" for example
spx_distance_weight = 1.00
floor_weight = 1.00
house_weight = 0.56
business_weight = 0.34
cvs_weight = 1.00
train_weight = 1.00
bus_weight = 1.00
radius = 300
businessList = [ "公車站", "火車站", "CVS"]

dfFinalPartnerShop = calculateTotalScore(dfFinalPartnerShop, \
            spx_distance_weight,\
            floor_weight,\
            house_weight,\
            business_weight, \
            cvs_weight, \
            train_weight,\
            bus_weight,\
            radius,businessList )
dfFinalPartnerShop

,brand,store,city_district,city,district,village,road,nearest_spx,distance(m),distance_score,...,公車站_500,火車站_500,CVS_500,公車站_600,火車站_600,CVS_600,latitude,longitude,address,total_score
0,Tomods,北投石牌二店,台北市北投區,臺北市,北投區,永明里,石牌路,北投石牌店,477.08,2.0,...,0.558333,0.030303,0.172727,0.666667,0.029412,0.000000,25.117289,121.516357,台北市北投區永明里3鄰石牌路二段93號,2.113504
1,Tomods,捷運內湖店,台北市內湖區,臺北市,內湖區,內湖里,成功路,內湖金龍店,289.83,1.0,...,0.591667,0.030303,0.300000,0.633333,0.029412,0.300000,25.084081,121.594547,台北市內湖區內湖里25鄰成功路四段188號,1.829060
2,Tomods,三井林口店,新北市林口區,新北市,林口區,南勢里,文化三路,林口民族店,326.29,1.0,...,0.758333,0.075758,0.272727,0.950000,0.088235,0.500000,25.071241,121.363547,新北市林口區南勢里22鄰文化三路一段356號,2.197265
3,Tomods,中山五常店,台北市中山區,臺北市,中山區,行仁里,五常街,松山復興店,530.21,2.0,...,0.550000,0.000000,0.181818,0.633333,0.000000,0.333333,25.064402,121.539213,台北市中山區行仁里1鄰五常街47號,2.227949
4,Tomods,南港中信店,台北市南港區,臺北市,南港區,三重里,經貿二路,南港忠孝店,719.28,2.0,...,0.483333,0.212121,0.172727,0.633333,0.294118,0.316667,25.059222,121.615185,台北市南港區三重里13鄰經貿二路186號,2.034444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3462,蝦皮店到店,尚無店名,NaN,桃園市,中壢區,福安里,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,24.956627,121.210713,"1樓, No. 277號, Section 1, Fuxing E Rd, Zhubei C...",NaN
3463,蝦皮店到店,台南民權店,NaN,臺南市,中西區,三民里,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,22.993952,120.206516,"1樓, No. 12號, Section 2, Minquan Rd, West Centr...",NaN
3464,亞太電信,八德桃鶯店,NaN,桃園市,八德區,雲林里,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,24.970025,121.327900,"No. 106號, Taoying Rd, Bade District, Taoyuan C...",NaN
3465,蝦皮店到店,伸港信義店,NaN,彰化縣,伸港鄉,南社里,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,24.261840,120.552336,"509, Taiwan, Changhua County, Shengang Townshi...",NaN


## 單一合作夥伴店址分數計算

### Step 1 爬蟲門市

以健身工廠為例

In [ ]:
store = []
id_list = []

# 爬取前五間店的店名
for i in range(0, 5):
    url = f"https://www.fitnessfactory.com.tw/findLocationsName?id={i}"
    re = requests.get(url)
    if re.status_code == 200:
        print("Crawling", url)
        
    # 將爬取的資料轉成 json 格式
    data = re.json()
    
    for d in data:
        id_list.append(d["id"])
        store.append(d["location"])
len(store) == len(id_list)


Crawling https://www.fitnessfactory.com.tw/findLocationsName?id=0
Crawling https://www.fitnessfactory.com.tw/findLocationsName?id=1
Crawling https://www.fitnessfactory.com.tw/findLocationsName?id=2
Crawling https://www.fitnessfactory.com.tw/findLocationsName?id=3
Crawling https://www.fitnessfactory.com.tw/findLocationsName?id=4


True

In [ ]:
address = []
lat = []
lng = []
location = []

# 爬取各店點的地址、經緯度
for i in id_list:
    url = f"https://www.fitnessfactory.com.tw/findGoogleMapLocations?id={i}"
    if i % 5 == 0:
        print("sleep a while")
        time.sleep(2)
    
    re = requests.get(url)
    print(f"Crawling:", url)
    data = re.json()

    address.append(data[0]["address"])
    lat.append(data[0]["lat"])
    lng.append(data[0]["lng"])
    location.append(data[0]["location"])
print(len(id_list))
print(len(address))

Crawling: https://www.fitnessfactory.com.tw/findGoogleMapLocations?id=3
Crawling: https://www.fitnessfactory.com.tw/findGoogleMapLocations?id=4
sleep a while
Crawling: https://www.fitnessfactory.com.tw/findGoogleMapLocations?id=5
Crawling: https://www.fitnessfactory.com.tw/findGoogleMapLocations?id=6
Crawling: https://www.fitnessfactory.com.tw/findGoogleMapLocations?id=7
Crawling: https://www.fitnessfactory.com.tw/findGoogleMapLocations?id=8
Crawling: https://www.fitnessfactory.com.tw/findGoogleMapLocations?id=9
Crawling: https://www.fitnessfactory.com.tw/findGoogleMapLocations?id=18
sleep a while
Crawling: https://www.fitnessfactory.com.tw/findGoogleMapLocations?id=35
Crawling: https://www.fitnessfactory.com.tw/findGoogleMapLocations?id=43
Crawling: https://www.fitnessfactory.com.tw/findGoogleMapLocations?id=46
Crawling: https://www.fitnessfactory.com.tw/findGoogleMapLocations?id=47
sleep a while
Crawling: https://www.fitnessfactory.com.tw/findGoogleMapLocations?id=50
Crawling: https:

In [ ]:
# 將爬蟲得來的資訊，透過 dataframe 整合，包含店址名稱、地址
DataFrame = {
    "store": store,
    "brand" :"健身工廠",
    "address": address,
    
}

file_name = f"健身工廠"
df = pd.DataFrame(DataFrame)

# 將 dataframe 輸出成 excel file
df.to_excel(f"門店資料_{file_name}.xlsx", index=False)

### Step 2


門市資料 store、	brand	、address 有這三個欄位


In [ ]:
#可以換成其他檔案
dfPartner = pd.read_excel("./門店資料_健身工廠.xlsx")

In [ ]:
#Step 2

# 設定爬蟲的 token, cookie, 與隨機生成 user-agent
token = "U65yWjl1zDnIAUMdC3gzynbXJHICvDvD6h_ekfwAst9FpcuKi3t96K6bvz8eT-0ze2CTvTah7tMzMd-a_EQREw.."
cookie = 'ExtentJS=25.047573%2C121.544425%2C18; _ga=GA1.3.1959134994.1667926043; ASP.NET_SessionId=3te0ywcl4gd40rxstpjzdwny'
rowNum = len(dfPartner)
ua = UserAgent()  
# 爬取各店址的營建署貼標，如是商業區，還是住宅區
dfNewAddressTag = findFullAddress(dfPartner,'address',token,cookie,0,rowNum-1)
# 以 csv file 輸出各店址的土地利用標籤
dfNewAddressTag.to_csv(f'./dfNewAddressTag.csv') 



0
Step1-Crawling: https://nsp.tcd.gov.tw/tcdgm/ws_data.ashx?&CMD=GETADDRESS&VAL=左營區博愛三路102號&COUNTY=高雄市
Step2-Crawling: https://nsp.tcd.gov.tw/tcdgm/ws_identify.ashx?token=U65yWjl1zDnIAUMdC3gzynbXJHICvDvD6h_ekfwAst9FpcuKi3t96K6bvz8eT-0ze2CTvTah7tMzMd-a_EQREw..&LAYERS=5%2C3%2C6&X=120.305192&Y=22.673611&Z=TWS
1 ['第五種住宅區']
1
Step1-Crawling: https://nsp.tcd.gov.tw/tcdgm/ws_data.ashx?&CMD=GETADDRESS&VAL=三民區慶雲街1號&COUNTY=高雄市
Step2-Crawling: https://nsp.tcd.gov.tw/tcdgm/ws_identify.ashx?token=U65yWjl1zDnIAUMdC3gzynbXJHICvDvD6h_ekfwAst9FpcuKi3t96K6bvz8eT-0ze2CTvTah7tMzMd-a_EQREw..&LAYERS=5%2C3%2C6&X=120.314948&Y=22.64072&Z=TWS
2 ['第五種住宅區', '第三種特定商業專用區']
2
Step1-Crawling: https://nsp.tcd.gov.tw/tcdgm/ws_data.ashx?&CMD=GETADDRESS&VAL=前鎮區文橫三路59號&COUNTY=高雄市
Step2-Crawling: https://nsp.tcd.gov.tw/tcdgm/ws_identify.ashx?token=U65yWjl1zDnIAUMdC3gzynbXJHICvDvD6h_ekfwAst9FpcuKi3t96K6bvz8eT-0ze2CTvTah7tMzMd-a_EQREw..&LAYERS=5%2C3%2C6&X=120.306921&Y=22.613606&Z=TWS
3 ['第五種住宅區', '第三種特定商業專用區', '第四種商業區']
3
St

建議貼完區域特型從貼標開始

In [ ]:
# 讀取 土地利用標籤的 csv file
dfNewAddressTag = pd.read_csv("./dfNewAddressTag.csv")
dfNewAddressTag

In [ ]:
dfNewAddressTag['floor'] = dfNewAddressTag.apply(judgeFloor, axis=1)
dfNewAddressTag['city_district'] = dfNewAddressTag['city'] + dfNewAddressTag['district']
dfNewAddressTag.rename(columns={"lng_list": "longitude", "lat_list": "latitude"}, inplace = True)
dfNewAddressTag = pd.merge(dfNewAddressTag, dfPartner, left_index=True,right_index=True)
dfNewPartnerStore = solveReplaceDistrict(dfNewAddressTag) 
dfNewPartnerStore.rename(columns={"住宅區": "住宅區分數", "其他區": "其他區分數","商業區": "商業區分數","1F": "是否一樓"}, inplace = True)
dfNewPartnerStore.to_csv(f'./dfNewPartnerStore.csv') 


### Step 3、4、5

In [ ]:
### Step 345
#Step 3 計算旅次密度
dfCarrierPop = pd.read_excel("./DataSet/DAC_台灣各時段電信人口資料.xlsx")
dfPopDensity = pd.read_excel("./DataSet/DAC_人口密度.xlsx")
dfTravelDensity = TripDensity(df1=dfCarrierPop,
                   city="COUNTY",
                   district="TOWN",
                   afternoon_work="AFTERNOON_WORK",
                   df2=dfPopDensity,
                   city_district="區域別",
                   area="土地面積")
#Step 4 計算人口密度
dfPopDensity_CC = PopulationDensity(df=dfPopDensity, city_district="區域別", population_density="人口密度")
dfPopDensity_CC
#Step 5 計算供不應求
dfCarrierPop2 = pd.read_excel("./DataSet/DAC_台灣各時段電信人口資料.xlsx")
dfSpxStore = pd.read_excel("./DataSet/DAC_蝦皮店到店地理資料_20221012.xlsx")
dfExcessDemand = ExcessDemand(df1=dfCarrierPop2, trip_city="COUNTY", trip_district="TOWN",
                  evening_work_trip="EVENING_WORK", df2=dfSpxStore, alias="alias", spx_city="city",
                  spx_district="district")
# 加入 merge 人口密度、供不應求、旅次密度
dfExcessDemand = pd.merge(dfExcessDemand, dfPopDensity_CC, how="left", on=["city_district"])
dfExcessDemand = pd.merge(dfExcessDemand, dfTravelDensity, how="left", on=["city_district"])
dfExcessDemand["city_district"]  = dfExcessDemand["city_district"].apply(lambda x: x.replace("台","臺"))
dfExcessDemand["city_district"] = dfExcessDemand["city_district"].str.strip()
dfNewPartnerStore["city_district"]  = dfNewPartnerStore["city_district"].apply(lambda x: x.replace("台","臺"))
dfNewPartnerStore["city_district"] = dfNewPartnerStore["city_district"].str.strip()
dfNewPartnerStore = pd.merge(dfNewPartnerStore, dfExcessDemand, how="left", on=["city_district"])
#用中位數補null值
dfNewPartnerStore["excess_demand_trip"].fillna(dfNewPartnerStore['excess_demand_trip'].median(), inplace = True)
dfNewPartnerStore["population_density"].fillna(dfNewPartnerStore['population_density'].median(), inplace = True)
dfNewPartnerStore["trip_density"].fillna(dfNewPartnerStore['trip_density'].median(), inplace = True)

In [ ]:
# Step 6 計算3D分群
df3DAnalysis_Partner = ThreeDimension(df=dfNewPartnerStore, trip_density="trip_density", population_density="population_density", excess_demand_trip="excess_demand_trip")
df3DAnalysis_Partner
# Step 6 計算夥伴與蝦皮店到店距離
dfSpx = pd.read_excel("./DataSet/DAC_蝦皮店到店地理資料_20221012.xlsx")
dfNearStorePartner = calculateCloestDistanceBetweenPoints(df3DAnalysis_Partner,dfSpx)
dfFinalPartner = pd.merge(df3DAnalysis_Partner, dfNearStorePartner, how="left", on=["brand","store","latitude","longitude"])

Now 2022-12-31, Calculating
正在計算 合作夥伴: 第1間店址
左營重上店 272.35
正在計算 合作夥伴: 第2間店址
三民建工店 1600.8
正在計算 合作夥伴: 第3間店址
前鎮三多店 944.61
正在計算 合作夥伴: 第4間店址
新興八德店 1417.04
正在計算 合作夥伴: 第5間店址
三民覺民店 1534.17
正在計算 合作夥伴: 第6間店址
小港港信店 1997.55
正在計算 合作夥伴: 第7間店址
左營實踐店 11784.71
正在計算 合作夥伴: 第8間店址
鳳山工協店 14389.56
正在計算 合作夥伴: 第9間店址
新興八德店 1015.78
正在計算 合作夥伴: 第10間店址
鼓山西藏店 918.67
正在計算 合作夥伴: 第11間店址
左營實踐店 4438.74
正在計算 合作夥伴: 第12間店址
三民覺民店 1165.34
正在計算 合作夥伴: 第13間店址
鳳山過勇店 18331.65
正在計算 合作夥伴: 第14間店址
左營實踐店 10110.77
正在計算 合作夥伴: 第15間店址
新興八德店 1055.03
正在計算 合作夥伴: 第16間店址
嘉義民族店 130.79
正在計算 合作夥伴: 第17間店址
台南中華店 768.15
正在計算 合作夥伴: 第18間店址
台南裕文店 449.28
正在計算 合作夥伴: 第19間店址
台南開元店 177.84
正在計算 合作夥伴: 第20間店址
民雄建國店 23815.75
正在計算 合作夥伴: 第21間店址
安平永華店 265.11
正在計算 合作夥伴: 第22間店址
安南海佃二店 523.36
正在計算 合作夥伴: 第23間店址
民雄建國店 4435.47
正在計算 合作夥伴: 第24間店址
台南生產店 253.3
正在計算 合作夥伴: 第25間店址
南屯大業店 518.56
正在計算 合作夥伴: 第26間店址
豐原圓環二店 305.78
正在計算 合作夥伴: 第27間店址
台中忠孝店 380.54
正在計算 合作夥伴: 第28間店址
台中中清二店 265.62
正在計算 合作夥伴: 第29間店址
台中工學店 507.03
正在計算 合作夥伴: 第30間店址
北屯同榮店 839.46
正在計算 合作夥伴: 第31間店址
員林林森店 940.17


### Step 7、8

In [ ]:
# Step 7 計算附加效益
#type you google api key
API_KEY = ''
#calculate_additional_benefit(dataframe start, dataframe end, radius distance, store_type, google API_key, data)
# the number of [ "學校", "公車站", "火車站","便利商店"]  

# 設定要爬取計算額外效益的門路類型
ltShoptype = [ "公車站", "火車站","便利商店"]
# 設定以多少公尺為半徑計算門路類型的個數
radius_list =[100,200,300,400,500,600] 

# 從半徑 100 公尺開始，爬取 shoptype 的數量，到半徑 600 公尺結束
for radius in radius_list:
  for index,i in enumerate(ltShoptype) :
    dfStoreNum = calculate_additional_benefit( 0,len(dfFinalPartner), radius, i,API_KEY, dfFinalPartner)
    dfFinalPartner = pd.merge(dfFinalPartner, dfStoreNum, how="left", on=["brand","store","latitude","longitude"])

# 將夥伴額外效益的資料集匯出
dfFinalPartner.to_csv(f'./dfFinalPartner.csv')


Now 2022-12-31, Calculating
Crawling 公車站, Index: 0
Now sleep for 7 sec
Crawling 公車站, Index: 1
Crawling 公車站, Index: 2
Crawling 公車站, Index: 3
Crawling 公車站, Index: 4
Crawling 公車站, Index: 5
Crawling 公車站, Index: 6
Crawling 公車站, Index: 7
Crawling 公車站, Index: 8
Crawling 公車站, Index: 9
Crawling 公車站, Index: 10
Crawling 公車站, Index: 11
Crawling 公車站, Index: 12
Crawling 公車站, Index: 13
Crawling 公車站, Index: 14
Crawling 公車站, Index: 15
Crawling 公車站, Index: 16
Crawling 公車站, Index: 17
Crawling 公車站, Index: 18
Crawling 公車站, Index: 19
Crawling 公車站, Index: 20
Now sleep for 5 sec
Crawling 公車站, Index: 21
Crawling 公車站, Index: 22
Crawling 公車站, Index: 23
Crawling 公車站, Index: 24
Crawling 公車站, Index: 25
Crawling 公車站, Index: 26
Crawling 公車站, Index: 27
Crawling 公車站, Index: 28
Crawling 公車站, Index: 29
Crawling 公車站, Index: 30
Crawling 公車站, Index: 31
Crawling 公車站, Index: 32
Crawling 公車站, Index: 33
Crawling 公車站, Index: 34
Crawling 公車站, Index: 35
Crawling 公車站, Index: 36
Crawling 公車站, Index: 37
Crawling 公車站, Index: 38
Crawli

In [ ]:
#Step 8
from sklearn.preprocessing import MinMaxScaler
# addtional_values = ['公車站_100', '火車站_100',
#        '便利商店_100', '公車站_200', '火車站_200', '便利商店_200', '公車站_300', '火車站_300',
#        '便利商店_300', '公車站_400', '火車站_400', '便利商店_400', '公車站_500', '火車站_500',
#        '便利商店_500', '公車站_600', '火車站_600', '便利商店_600']

# 將爬取獲得的 公車站、便利商店、火車站數量進行標準化
addtional_values = dfFinalPartner.columns[-18:]
X = dfFinalPartner[addtional_values] 
# 將標準化的數值設定在 0 - 1 之間
scaler = MinMaxScaler(feature_range=(0, 1)).fit(X)
X_scaled = scaler.transform(X)
dfFinalPartner[addtional_values] = X_scaled

# 設定計算合作店址總分的權重
# take the radius "300m" for example
spx_distance_weight = 1.00
floor_weight = 1.00
house_weight = 0.56
business_weight = 0.34
cvs_weight = 1.00
train_weight = 1.00
bus_weight = 1.00
radius = 300
businessList = [ "公車站", "火車站","便利商店"]

dfFinal_Partner = calculateTotalScore(dfFinalPartner, \
            spx_distance_weight,\
            floor_weight,\
            house_weight,\
            business_weight, \
            cvs_weight, \
            train_weight,\
            bus_weight,\
            radius,businessList )
dfFinal_Partner.to_csv(f'./dfFinal_Partner.csv')
dfFinal_Partner

,city,road,label,full_add,district,village,longitude,latitude,floor,city_district,...,火車站_400,便利商店_400,公車站_500,火車站_500,便利商店_500,公車站_600,火車站_600,便利商店_600,distance_score,total_score
0,高雄市,博愛三路,第五種住宅區,高雄市左營區菜公里2鄰博愛三路102號,左營區,菜公里,120.305192,22.673611,1F,高雄市左營區,...,0.000000,0.379310,0.086207,0.000000,0.457143,0.100000,0.000000,0.000,1,2.207528
1,高雄市,慶雲街,第三種特定商業專用區,高雄市三民區安吉里6鄰慶雲街1號,三民區,安吉里,120.314948,22.640720,1F,高雄市三民區,...,0.105263,0.310345,0.086207,0.111111,0.400000,0.183333,0.111111,0.450,3,1.774875
2,高雄市,文橫三路,第四種商業區,高雄市前鎮區盛興里6鄰文橫三路59號,前鎮區,盛興里,120.306921,22.613606,1F,高雄市前鎮區,...,0.052632,0.586207,0.241379,0.000000,0.600000,0.266667,0.111111,0.500,3,1.759192
3,高雄市,五福二路,第五種商業區,高雄市新興區玉衡里10鄰五福二路262號,新興區,玉衡里,120.301689,22.623557,1F,高雄市新興區,...,0.052632,0.275862,0.172414,0.055556,0.314286,0.250000,0.055556,0.425,3,1.840532
4,高雄市,澄清路,第四種住宅區,高雄市三民區寶業里23鄰澄清路466之1號,三民區,寶業里,120.346073,22.641582,1F,高雄市三民區,...,0.000000,0.413793,0.327586,0.000000,0.457143,0.333333,0.055556,0.475,3,1.923636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,臺北市,遼寧街,第三種商業區(特),臺北市中山區復華里11鄰遼寧街177號,中山區,復華里,121.542365,25.052796,1F,臺北市中山區,...,0.052632,0.000000,0.275862,0.055556,0.857143,0.466667,0.055556,0.975,1,2.668283
60,新北市,龍安路,住宅區,新北市新莊區龍安里15鄰龍安路252之1號,新莊區,龍安里,121.421018,25.019438,1F,新北市新莊區,...,0.000000,0.586207,0.258621,0.000000,0.828571,0.300000,0.000000,0.500,0,2.553620
61,新北市,中山二路,乙種工業區,新北市蘆洲區中華里2鄰中山二路162號,蘆洲區,中華里,121.470990,25.081188,1F,新北市蘆洲區,...,0.000000,0.793103,0.603448,0.000000,0.800000,0.616667,0.000000,0.900,0,2.893429
62,新北市,文化二路,中心商業區,新北市林口區湖南里10鄰文化二路二段1號,林口區,湖南里,121.376434,25.079055,1F,新北市林口區,...,0.000000,0.758621,0.689655,0.000000,0.714286,0.916667,0.000000,0.650,1,2.977959
